# Homework #3 - Rossman Store Sales

Group Members:

Hoo De Lin (A0126576X)

Yap Han Chiang (A0125168E)

# Analyzing the data:

Before we try figuring out which model to fit in, we need to understand the data, and clean/modify the data to be able to feed into any model

The feature that I believe are quite important are:

- Competition
- Holidays
- Type of stores
- Type of day (Monday - Sunday)
- Having promotions
- Promo2 since

Interesting to note is that, 

In [2]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import matplotlib.pyplot as pl
pl.style.use('ggplot')
%matplotlib inline
import seaborn as sb 
import csv

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
trainData = pd.read_csv('train_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str'})
testData = pd.read_csv('test_v2.csv',  parse_dates=['Date'], dtype={'StateHoliday': 'str'})
storeData = pd.read_csv('store.csv')

trainData = pd.merge(storeData, trainData, on='Store')
testData = pd.merge(storeData, testData, on='Store')
# testDataUnsanitized = pd.read_csv('test_v2.csv',  parse_dates=['Date'], dtype={'StateHoliday': 'str'})

In [12]:
# Cleaning the data
def hasHoliday(row):
    if row['StateHoliday'] != "0" or row['SchoolHoliday'] != "0":
        return 1
    else:
        return 0
def calculateCompetitionAge(row):
        if math.isnan(row['CompetitionOpenSinceMonth']):
            return 0
        else:
            f_date = datetime(int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']), 1, 0, 0, 0)
            return (row['Date'].to_pydatetime() - f_date).days

month_num_name_dict = {
    "1": "Jan",
    "2": "Feb",
    "3": "Mar",
    "4": "Apr",
    "5": "May",
    "6": "Jun",
    "7": "Jul",
    "8": "Aug",
    "9": "Sept",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec"
}

def isHavingPromo2(row):
    if not isinstance(row['PromoInterval'], str):
        return 0
    elif month_num_name_dict[str(row['Date'].to_pydatetime().month)] in row['PromoInterval']:
        return 1
    else:
        return 0

# Possible thought: If we are not at the promo interval, should promo2Age have any effect?
def promo2Age(row):
    if row['Promo2'] == 0:
        return 0
    else:
#        https://stackoverflow.com/a/17087427
        date = str(int(row["Promo2SinceYear"])) + "-W" + str(int(row["Promo2SinceWeek"]))
        f_date = datetime.strptime(date + '-0', "%Y-W%W-%w")
        return (row['Date'].to_pydatetime() - f_date).days


In [13]:
trainData['HasHoliday'] = trainData.apply(lambda row: hasHoliday(row), axis=1)
trainData['CompetitionAge'] = trainData.apply(lambda row: calculateCompetitionAge(row), axis=1)
trainData['HavingPromo2'] = trainData.apply(lambda row: isHavingPromo2(row), axis=1)
trainData['Promo2Age'] = trainData.apply(lambda row: promo2Age(row), axis=1)

In [14]:
trainData.drop('CompetitionOpenSinceMonth', axis=1, inplace=True)
trainData.drop('CompetitionOpenSinceYear', axis=1, inplace=True)
trainData.drop('StateHoliday', axis=1, inplace=True)
trainData.drop('SchoolHoliday', axis=1, inplace=True)
trainData.drop('Promo2SinceWeek', axis=1, inplace=True)
trainData.drop('Promo2SinceYear', axis=1, inplace=True)
trainData.drop('PromoInterval', axis=1, inplace=True)
trainData.drop('Promo2', axis=1, inplace=True)
trainData.drop('Open', axis=1, inplace=True)
trainDataSanitizedStore = trainData.loc[:, 'Store']
trainData.drop('Store', axis=1, inplace=True)

In [15]:
trainData

,StoreType,Assortment,CompetitionDistance,DayOfWeek,Date,Sales,Customers,Promo,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,c,a,1270.0,5,2015-01-30,5577,616,1,1,2342,0,0
1,c,a,1270.0,4,2015-01-29,5171,573,1,1,2341,0,0
2,c,a,1270.0,3,2015-01-28,4310,537,1,1,2340,0,0
3,c,a,1270.0,2,2015-01-27,4806,532,1,1,2339,0,0
4,c,a,1270.0,1,2015-01-26,4781,566,1,1,2338,0,0
5,c,a,1270.0,7,2015-01-25,0,0,0,1,2337,0,0
6,c,a,1270.0,6,2015-01-24,4840,600,0,1,2336,0,0
7,c,a,1270.0,5,2015-01-23,3586,489,0,1,2335,0,0
8,c,a,1270.0,4,2015-01-22,3492,493,0,1,2334,0,0
9,c,a,1270.0,3,2015-01-21,3299,460,0,1,2333,0,0


In [16]:
testData['HasHoliday'] = testData.apply(lambda row: hasHoliday(row), axis=1)
testData['CompetitionAge'] = testData.apply(lambda row: calculateCompetitionAge(row), axis=1)
testData['HavingPromo2'] = testData.apply(lambda row: isHavingPromo2(row), axis=1)
testData['Promo2Age'] = testData.apply(lambda row: promo2Age(row), axis=1)

In [18]:
testData.drop('CompetitionOpenSinceMonth', axis=1, inplace=True)
testData.drop('CompetitionOpenSinceYear', axis=1, inplace=True)
testData.drop('StateHoliday', axis=1, inplace=True)
testData.drop('SchoolHoliday', axis=1, inplace=True)
testData.drop('Promo2SinceWeek', axis=1, inplace=True)
testData.drop('Promo2SinceYear', axis=1, inplace=True)
testData.drop('PromoInterval', axis=1, inplace=True)
testData.drop('Promo2', axis=1, inplace=True)
testData.drop('Open', axis=1, inplace=True)
testDataSanitizedStore = testData.loc[:, 'Store']
testData.drop('Store', axis=1, inplace=True)

In [19]:
testData

,StoreType,Assortment,CompetitionDistance,DayOfWeek,Date,Customers,Promo,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,c,a,1270.0,5,2015-07-31,555,1,1,2524,0,0
1,c,a,1270.0,4,2015-07-30,546,1,1,2523,0,0
2,c,a,1270.0,3,2015-07-29,523,1,1,2522,0,0
3,c,a,1270.0,2,2015-07-28,560,1,1,2521,0,0
4,c,a,1270.0,1,2015-07-27,612,1,1,2520,0,0
5,c,a,1270.0,7,2015-07-26,0,0,1,2519,0,0
6,c,a,1270.0,6,2015-07-25,500,0,1,2518,0,0
7,c,a,1270.0,5,2015-07-24,459,0,1,2517,0,0
8,c,a,1270.0,4,2015-07-23,503,0,1,2516,0,0
9,c,a,1270.0,3,2015-07-22,463,0,1,2515,0,0


In [532]:
# Technically we don't need this anymore lol

# 1. Replace all NaN values with 0
# 2. Map all character to integers

# For mapping StateHoliday, StoreType, Assortment to integers
mapCharToInt = {'0': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4}
mapPromoIntervalToInt = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7,
                        'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
def clean_data():
    storeDataUnsanitized.fillna(0, inplace=True)
    trainData.fillna(0, inplace=True)
    testData.fillna(0, inplace=True)
    
    storeDataUnsanitized.replace(mapCharToInt, inplace=True)
    trainData.replace(mapCharToInt, inplace=True)
    testData.replace(mapCharToInt, inplace=True)
    return (storeDataUnsanitized, trainData, testData)

In [533]:
storeDataSanitized, trainDataSanitized, testDataSanitized = clean_data()
trainDataSanitized.head()

,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,hasHoliday,CompetitionAge,havingPromo2
0,3,1,1270.0,0,0.0,0.0,0,5,2015-01-30,5577,616,1,1,1,2342,0
1,3,1,1270.0,0,0.0,0.0,0,4,2015-01-29,5171,573,1,1,1,2341,0
2,3,1,1270.0,0,0.0,0.0,0,3,2015-01-28,4310,537,1,1,1,2340,0
3,3,1,1270.0,0,0.0,0.0,0,2,2015-01-27,4806,532,1,1,1,2339,0
4,3,1,1270.0,0,0.0,0.0,0,1,2015-01-26,4781,566,1,1,1,2338,0


In [534]:
testDataSanitized.head()

,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Customers,Open,Promo,hasHoliday,CompetitionAge,havingPromo2
0,3,1,1270.0,0,0.0,0.0,0,5,2015-07-31,555,1,1,1,2524,0
1,3,1,1270.0,0,0.0,0.0,0,4,2015-07-30,546,1,1,1,2523,0
2,3,1,1270.0,0,0.0,0.0,0,3,2015-07-29,523,1,1,1,2522,0
3,3,1,1270.0,0,0.0,0.0,0,2,2015-07-28,560,1,1,1,2521,0
4,3,1,1270.0,0,0.0,0.0,0,1,2015-07-27,612,1,1,1,2520,0


In [535]:
# Save some data, drop some columns
if 'Date' in trainDataSanitized.columns:
    trainDataSanitizedDate = trainDataSanitized.loc[:, 'Date']
    trainDataSanitized.drop('Date', axis=1, inplace=True)
if 'PromoInterval' in trainDataSanitized.columns:
    trainDataSanitizedPromoInterval = trainDataSanitized.loc[:, 'PromoInterval']
    trainDataSanitized.drop('PromoInterval', axis=1, inplace=True)
if 'Sales' in trainDataSanitized:
    trainDataSanitizedLabel = trainDataSanitized.loc[:, 'Sales']
    trainDataSanitized.drop('Sales', axis=1, inplace=True)
    
if 'Date' in testDataSanitized.columns:
    testDataSanitizedDate = testDataSanitized.loc[:, 'Date']
    testDataSanitizedDate = testDataSanitizedDate.apply(lambda a: a.strftime("%Y-%m-%d"))
    testDataSanitized.drop('Date', axis=1, inplace=True)
if 'PromoInterval' in testDataSanitized.columns:
    testDataSanitizedPromoInterval = testDataSanitized.loc[:, 'PromoInterval']
    testDataSanitized.drop('PromoInterval', axis=1, inplace=True)


In [536]:
model = XGBRegressor().fit(trainDataSanitized, trainDataSanitizedLabel)
prediction = model.predict(testDataSanitized)
print(prediction)

[ 5003.86816406  4956.06835938  4777.61376953 ...,  5411.32177734
  6632.65576172  6349.30712891]


In [587]:
def dateStringToInt(dateString):
    (year, month, day) = dateString.split('-')
    result = int(year + month + day)
    return result

In [588]:
testDataSanitizedInt = testDataSanitizedDate.apply(lambda x: dateStringToInt(x))

In [609]:
def writeResultToFile(result):
    with open('submission.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['Id', 'Sales'])
        rows = list(zip(testDataSanitizedStore, testDataSanitizedInt))
        for i in range(prediction.shape[0]):
            row = rows[i]
            # print(type(row[0]), type(row[1]))
            # writer.writerow({'Id': row, 'Sales': prediction[i]})
            writer.writerow([row, prediction[i]])

In [621]:
# print(trainDataSanitizedDate[0].strftime("%Y-%m-%d"))
# print(testDataSanitizedStore.shape) == 33450
# print(prediction.shape) == 33450
# sales = prediction[0]
# ident = testDataSanitizedStore.iloc[i]
a = list(zip(testDataSanitizedStoreInt, testDataSanitizedInt))
print(a[0])

# writeResultToFile(prediction)

(1, 20150731)


In [627]:
with open('submission.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    i = 0
    for row in reader:
        if i == 10:
            break
        print(row)
        i += 1

['Id', 'Sales']
['(1, 20150731)', '5003.87']
['(1, 20150730)', '4956.07']
['(1, 20150729)', '4777.61']
['(1, 20150728)', '5268.43']
['(1, 20150727)', '6148.67']
['(1, 20150726)', '-269.62']
['(1, 20150725)', '4192.24']
['(1, 20150724)', '3674.13']
['(1, 20150723)', '4071.19']
